## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns = ["EIN", "NAME"], inplace=True)

In [4]:
# Determine the number of unique values in each column.
unique = application_df.dtypes.index.tolist()
application_df[unique].nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
app_type = application_df["APPLICATION_TYPE"].value_counts()
app_type

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type[app_type < 500].index)
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
class_count = application_df["CLASSIFICATION"].value_counts()

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_count[class_count >1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_count[class_count < 100].index)
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
num_df = pd.get_dummies(application_df)

In [11]:
# Split our preprocessed data into our features and target arrays
X = num_df.drop(columns="IS_SUCCESSFUL").values
y = num_df["IS_SUCCESSFUL"].values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=50)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_feat = len(X_train_scaled[0])
layer1 = 10
layer2 = 30
layer3 = 90
layer4 = 270

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = layer1,input_dim = input_feat, activation = "relu"))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units = layer2, activation = "relu"))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units = layer3,input_dim = input_feat, activation = "relu"))
# 4th hiddern layer
nn.add(tf.keras.layers.Dense(units = layer4,input_dim = input_feat, activation = "relu"))



In [31]:
# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 10)                500       
                                                                 
 dense_12 (Dense)            (None, 30)                330       
                                                                 
 dense_13 (Dense)            (None, 90)                2790      
                                                                 
 dense_14 (Dense)            (None, 270)               24570     
                                                                 
 dense_15 (Dense)            (None, 1)                 271       
                                                                 
Total params: 28,461
Trainable params: 28,461
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Compile the model
nn.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [33]:
# Train the model
trained_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5868 - accuracy: 0.7041
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5605 - accuracy: 0.7250
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5563 - accuracy: 0.7250
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5544 - accuracy: 0.7263
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5527 - accuracy: 0.7272
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5525 - accuracy: 0.7273
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5516 - accuracy: 0.7271
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5507 - accuracy: 0.7299
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5499 - accuracy: 0.7281
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5498 - accura

804/804 [==============================] - 2s 2ms/step - loss: 0.5365 - accuracy: 0.7368
Epoch 82/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5367 - accuracy: 0.7377
Epoch 83/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5365 - accuracy: 0.7368
Epoch 84/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5361 - accuracy: 0.7372
Epoch 85/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5362 - accuracy: 0.7369
Epoch 86/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5366 - accuracy: 0.7378
Epoch 87/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5359 - accuracy: 0.7378
Epoch 88/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5365 - accuracy: 0.7371
Epoch 89/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5361 - accuracy: 0.7382
Epoch 90/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5371 - accuracy: 

In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5576 - accuracy: 0.7375 - 473ms/epoch - 2ms/step
Loss: 0.557560920715332, Accuracy: 0.7374927401542664


In [35]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity2.h5")